In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import tensorflow as tf
from helper_functions import *
import pandas as pd
# importTensorflow(memory=4090)

2.13.0
1 Physical GPUs, 1 Logical GPUs


In [2]:
df = pd.read_csv("queries.csv")
df.head()

,String,Class
0,Rich and buttery delicacy recipes,foie_gras
1,Luxurious appetizer with duck liver,foie_gras
2,Indulgent gourmet dish preparation,foie_gras
3,Recipes for savory liver pâté,foie_gras
4,Delicate and decadent French cuisine ideas,foie_gras


In [3]:
d = set()
counter = -1
l = []
for thing in df.Class:
    if thing in d:
      l.append(counter)
    else:
        counter += 1
        d.add(thing)
        l.append(counter)

In [4]:
class_list = list(set(df.Class))

In [5]:
df['classEncode'] = l

In [6]:
df.head()

,String,Class,classEncode
0,Rich and buttery delicacy recipes,foie_gras,0
1,Luxurious appetizer with duck liver,foie_gras,0
2,Indulgent gourmet dish preparation,foie_gras,0
3,Recipes for savory liver pâté,foie_gras,0
4,Delicate and decadent French cuisine ideas,foie_gras,0


In [7]:
df = df.sample(frac = 1)
df.head()

,String,Class,classEncode
2048,Serving suggestions for dishes featuring mac a...,macaroni_and_cheese,82
872,Recipes for delicious rice dish with aromatic ...,risotto,35
1595,Creative variations of carbonara with garlic o...,spaghetti_carbonara,64
500,Easy-to-make dish with dough wrappers and flav...,gyoza,20
2398,Quick and easy custard recipes for satisfying ...,creme_brulee,97


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.String, df.classEncode, test_size=0.1, random_state=69)

In [9]:
len(X_train), len(X_test)

(2232, 248)

In [10]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64).prefetch(tf.data.AUTOTUNE)
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64).prefetch(tf.data.AUTOTUNE)

In [11]:
vocab_length = 25000
max_length = [len(i.split()) for i in df.String]

In [12]:
import numpy as np
max_length = int(np.percentile(max_length, 95))

In [13]:
#vectorization
text_vector = tf.keras.layers.TextVectorization(max_tokens=vocab_length,
                                                output_sequence_length=max_length)

In [14]:
text_vector.adapt(df.String)

In [15]:
text_vector.get_vocabulary()[:5]

['', '[UNK]', 'for', 'with', 'and']

In [16]:
text_vector(["Luxurious ingredient for gourmet dining"])

<tf.Tensor: shape=(1, 12), dtype=int64, numpy=
array([[ 199, 1126,    2,  151,  382,    0,    0,    0,    0,    0,    0,
           0]])>

In [17]:
# embedding
embedding = tf.keras.layers.Embedding(input_dim = vocab_length,
                                      input_length=max_length,
                                      output_dim=1024,
                                      mask_zero=True)

In [18]:
embed = embedding(text_vector(["Luxurious ingredient for gourmet dining"]))

In [19]:
embed

<tf.Tensor: shape=(1, 12, 1024), dtype=float32, numpy=
array([[[-0.02559385,  0.01419305, -0.02053919, ..., -0.04268744,
          0.03478425,  0.02344571],
        [ 0.03239026,  0.03955312,  0.03160869, ..., -0.01277212,
         -0.0043489 , -0.03991058],
        [-0.02334566, -0.01881214, -0.0244652 , ...,  0.03512721,
          0.02422229,  0.01638706],
        ...,
        [-0.00054482, -0.04704479, -0.02888717, ...,  0.02786562,
         -0.00452663,  0.03906851],
        [-0.00054482, -0.04704479, -0.02888717, ...,  0.02786562,
         -0.00452663,  0.03906851],
        [-0.00054482, -0.04704479, -0.02888717, ...,  0.02786562,
         -0.00452663,  0.03906851]]], dtype=float32)>

In [20]:
one_hot_encoded_train = tf.one_hot(y_train, depth=101)
one_hot_encoded_test = tf.one_hot(y_test, depth=101)

In [21]:
train_data_one_hot = tf.data.Dataset.from_tensor_slices((X_train, one_hot_encoded_train)).shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)
test_data_one_hot = tf.data.Dataset.from_tensor_slices((X_test, one_hot_encoded_test)).batch(64).prefetch(tf.data.AUTOTUNE)

In [22]:
# conv1d
inputs = tf.keras.layers.Input(shape=(1, ), dtype='string')
x = text_vector(inputs)
x = embedding(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True))(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(x)
# x = tf.keras.layers.Conv1D(256, kernel_size=3)(x)
# x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.GRU(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(class_list), activation='softmax')(x)

# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch : 3e-3/(1+epoch)**(2))

model = tf.keras.models.Model(inputs, outputs)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 12)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 12, 1024)          25600000  
                                                                 
 bidirectional (Bidirection  (None, 12, 1024)          4724736   
 al)                                                             
                                                                 
 gru_1 (GRU)                 (None, 128)               443136    
                                                                 
 batch_normalization (Batch  (None, 128)               512   

In [23]:
history = model.fit(train_data_one_hot,
                    epochs=5,
                    validation_data=(test_data_one_hot),
                    validation_steps=int(0.25*len(test_data_one_hot)),
                    steps_per_epoch=len(train_data_one_hot))

Epoch 1/5


35/35 [==============================] - 11s 131ms/step - loss: 3.8165 - accuracy: 0.2173 - val_loss: 4.4219 - val_accuracy: 0.0312
Epoch 2/5
35/35 [==============================] - 2s 68ms/step - loss: 1.3518 - accuracy: 0.7312 - val_loss: 2.7954 - val_accuracy: 0.7031
Epoch 3/5
35/35 [==============================] - 1s 30ms/step - loss: 0.5501 - accuracy: 0.8696 - val_loss: 2.1713 - val_accuracy: 0.6875
Epoch 4/5
35/35 [==============================] - 1s 24ms/step - loss: 0.3013 - accuracy: 0.9270 - val_loss: 1.5088 - val_accuracy: 0.7188
Epoch 5/5
35/35 [==============================] - 1s 24ms/step - loss: 0.1942 - accuracy: 0.9516 - val_loss: 1.4669 - val_accuracy: 0.7500


In [24]:
model.evaluate(test_data_one_hot)

1/4 [======>.......................] - ETA: 0s - loss: 1.4669 - accuracy: 0.7500

4/4 [==============================] - 0s 7ms/step - loss: 1.5830 - accuracy: 0.7137


[1.5830291509628296, 0.7137096524238586]

In [25]:
# model.save("75_3_gru_one_hot/")

In [26]:
tf.argmax(model.predict(["Cooking tutorials for achieving the perfect macaron feet"]), axis=1)

1/1 [==============================] - 2s 2s/step


<tf.Tensor: shape=(1,), dtype=int64, numpy=array([55])>

In [27]:
class_list[75]

'beef_tartare'

# Pretrained embeddings

In [28]:
import tensorflow_hub as hub
embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")

In [29]:
embed(["Rice cooked food"])

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-0.02827955,  0.04139925,  0.0158745 ,  0.04136196,  0.05827901,
         0.03923946, -0.02325298,  0.00041435, -0.0523848 ,  0.05865195,
        -0.02272667, -0.06025485, -0.06244032, -0.04705139, -0.0854217 ,
         0.02652335, -0.01217343, -0.06420659,  0.00995875, -0.04074476,
         0.04196679,  0.07901078, -0.01970854, -0.01796671,  0.04481786,
         0.05687874, -0.04481113,  0.02043473, -0.07764187, -0.03506602,
         0.04480224,  0.06017666, -0.06590992, -0.00959308, -0.06850488,
         0.01169699, -0.01823763, -0.04149666,  0.04416808, -0.00070136,
         0.02090705, -0.01576048,  0.08526254, -0.06790678, -0.08336893,
         0.05019844, -0.00493785, -0.0217003 , -0.06885814, -0.08142836,
        -0.01130935,  0.05944003,  0.0343403 , -0.024447  ,  0.02204055,
         0.00451931, -0.0093224 ,  0.00847749,  0.07504914, -0.01754982,
        -0.01578058, -0.0539532 , -0.0403356 ,  0.03714855,  0.00210826,
 

In [30]:
sentence_encoder_layer = hub.KerasLayer(embed,
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=True)

In [31]:
model_1 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    # tf.keras.layers.Reshape((1, -1)),
    # tf.keras.layers.LSTM(256),
    # tf.keras.layers.Conv1D(64, kernel_size=3),
    # tf.keras.layers.GlobalAveragePooling1D,
    # tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(len(class_list), activation='softmax')
])
model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0035),
              metrics=['accuracy'])

In [32]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense_2 (Dense)             (None, 101)               51813     
                                                                 
Total params: 256849637 (979.80 MB)
Trainable params: 256849637 (979.80 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
history_1 = model_1.fit(train_data_one_hot,
                    epochs=5,
                    validation_data=(test_data_one_hot),
                    validation_steps=int(0.2*len(test_data_one_hot)),
                    steps_per_epoch=len(train_data_one_hot))

Epoch 1/5


35/35 [==============================] - 54s 1s/step - loss: 4.4436 - accuracy: 0.1891
Epoch 2/5
35/35 [==============================] - 10s 282ms/step - loss: 3.5828 - accuracy: 0.7496
Epoch 3/5
35/35 [==============================] - 6s 170ms/step - loss: 2.5403 - accuracy: 0.8898
Epoch 4/5
35/35 [==============================] - 6s 161ms/step - loss: 1.5885 - accuracy: 0.9597
Epoch 5/5
35/35 [==============================] - 4s 117ms/step - loss: 0.9004 - accuracy: 0.9848


In [34]:
model_1.evaluate(test_data_one_hot)

4/4 [==============================] - 0s 5ms/step - loss: 1.4923 - accuracy: 0.7661


[1.49229896068573, 0.7661290168762207]

In [35]:
# model_1.save("79_USE")